In [1]:
filename="./talk/common-Restaurant.md"
f = open(filename)
lines=f.readlines()
f.close()

In [16]:
raw='''* default "sorry, I didn't understand you, please try input something else"
'''
parts=raw[2:].split('"')
print(parts[0].strip())
print(parts[1])

default
sorry, I didn't understand you, please try input something else


In [42]:
import yaml

class IntentForm:
    def __init__(self):
        self.form_name="_unname_"
        self.lines=[]
    def print_form(self):        
        print(yaml.dump(self.get_form()))
    def get_form(self):
        document="".join(intent.form.lines)
        return yaml.load(document)

class IntentStory:
    def __init__(self):
        self.story_name="_unname_"
        self.lines=[]
    
class IntentSection:    
    def __init__(self, intent_name):
        self.intent_name=intent_name
        self.lines=[]
        self.attributes={}
        self.form=IntentForm()
        self.story=IntentStory()
        self.in_form=False
        self.in_story=False
    def add_intent_attr(self, attr_name, attr_cnt):
        self.attributes[attr_name]=attr_cnt
    def add_line(self, line):    
        if self.in_form and line.startswith("```"):
            self.in_form=False
            return                            
        if self.in_form:
            self.form.lines.append(line)
            return
        
        if self.in_story and not line.startswith("  "):
            self.in_story=False
        elif self.in_story:
            self.story.lines.append(line)
            return
            
        if line.startswith("* "):
            parts=line[2:].split('"')
            self.add_intent_attr(parts[0].strip(), parts[1])
        elif line.startswith("@ form "):
            self.form.form_name=line[len("@ form "):].strip()
        elif line.startswith("@ story "):
            self.story.story_name=line[len("@ story "):].strip()
            self.in_story=True
        elif line.startswith("```yaml"):
            self.in_form=True
        else:
            self.lines.append(line)


In [43]:
def parse_lines(lines):
    intents=[]
    intent=IntentSection("_head__")
    for line in lines:
        line_s=line.strip()
        if line.startswith("##"):        
            intent_name=line[3:].strip()
            print("☈", intent_name)
            intent=IntentSection(intent_name)
            intents.append(intent)
        elif len(line_s)==0 or line_s.startswith("#"):
            pass
        else:
            intent.add_line(line)
    return intents

def get_intent(intents, intent_name):
    it = next((x for x in intents if x.intent_name==intent_name), None)
    return it

intents=parse_lines(lines)

☈ request restaurant
☈ affirm restaurant
☈ inform


In [11]:
it = next((x for x in intents if x.intent_name=="affirm restaurant"), None)
print("".join(it.lines))

yeah a cheap restaurant serving international food



In [19]:
it = next((x for x in intents if x.intent_name=="request restaurant"), None)
print("".join(it.lines))

im looking for a restaurant
can i get [cuisine:o](swedish) food in any area
a restaurant that serves [cuisine:o](caribbean) food
id like a restaurant
im looking for a restaurant that serves [cuisine:o](mediterranean) food
can i find a restaurant that serves [cuisine:o](chinese)
i need to find a restaurant
uh im looking for a restaurant that serves [cuisine:o](kosher) food
uh can i find a restaurant and it should serve [cuisine:o](brazilian) food
im looking for a restaurant serving [cuisine:o](italian) food
restaurant please
i'd like to book a table for [num_people:n](two) with [cuisine:o](brazilian) cuisine
i need a table for [num_people:n](4)



In [23]:
intent=get_intent(intents, "request restaurant")
print(intent.form.form_name)
print(intent.attributes)
print("".join(intent.form.lines))

restaurant
{'default': "sorry, I didn't understand you, please try input something else", 'submit': 'done!'}
fields:
  cuisine:
    text: "what cuisine?"
  num_people:
    text: "how many people?"
    type: number
  outdoor_seating:
    text: "do you want to seat outside?"
    type: boolean
  preferences:
    text: "please provide additional preferences"
  feedback:
    text: "please give your feedback"



In [25]:
import yaml
document="".join(intent.form.lines)
print(yaml.dump(yaml.load(document)))

fields:
  cuisine: {text: 'what cuisine?'}
  feedback: {text: please give your feedback}
  num_people: {text: 'how many people?', type: number}
  outdoor_seating: {text: 'do you want to seat outside?', type: boolean}
  preferences: {text: please provide additional preferences}



In [40]:
intent=get_intent(intents, "request restaurant")
intent.form.print_form()
form=intent.form.get_form()
for fld in form["fields"]:
    print(fld)
fields=form["fields"]
print("num_people.type", "=", fields["num_people"]["type"])

fields:
  cuisine: {text: 'what cuisine?'}
  feedback: {text: please give your feedback}
  num_people: {text: 'how many people?', type: number}
  outdoor_seating: {text: 'do you want to seat outside?', type: boolean}
  preferences: {text: please provide additional preferences}

cuisine
num_people
outdoor_seating
preferences
feedback
num_people.type = number
